In [ ]:
import twitter
import os
from config import *
import time

# initialize api instance
twitter_api = twitter.Api(consumer_key=consumer_key,
                         consumer_secret=consumer_secret,
                         access_token_key=access_token_key,
                         access_token_secret=access_token_secret,
                        cache=None,tweet_mode='compat')

#print(consumer_key,consumer_secret,access_token_key,access_token_secret)

# test authentication
print(twitter_api.VerifyCredentials())
# print(twitter_api.GetStatus(126402758403305000))

In [ ]:
def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100,
                                               lang='en',result_type='recent')
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"text":status.text, "label":None, "user":status.user.screen_name, "time":status.created_at,
                "search_term":search_keyword} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

In [ ]:
# create blank testDataSet list
testDataSet = []

In [ ]:
# test search function
search_term = input("Enter a search keyword: ")
testDataSet.extend(buildTestSet(search_term))

In [ ]:
print(testDataSet[0:110])

# Use only to test twitter api

In [ ]:
tweets_fetched = twitter_api.GetSearch('disney', count = 100, lang='en')
[{"text":status.text, "label":None, "user":status.user.screen_name} for status in tweets_fetched]

# Start preprocessing of training data

In [ ]:
import pandas as pd
import numpy as np

traindata140 = os.path.join(".","trainingandtestdata_sentiment140","training.csv")

In [ ]:
#read the csv file command
train_df = pd.read_csv(traindata140, header=None, usecols=[0,5], names=['polarity of the tweet','text'], 
                       encoding="ISO-8859-1")
train_df.head()
# (0 = negative, 2 = neutral, 4 = positive)

# Use only to process the test dataset provided by Sentiment140

In [ ]:
testdata140 = os.path.join(".","trainingandtestdata_sentiment140","test.csv")
test_df = pd.read_csv(testdata140, header=None, usecols=[0,5],
                      names=['polarity of the tweet', 'text'],encoding="ISO-8859-1")
conditions_test = [
    (test_df['polarity of the tweet'] == 0),
    (test_df['polarity of the tweet'] == 2),
    (test_df['polarity of the tweet'] == 4)]
choices_test = ['negative', 'neutral', 'positive']
test_df['label'] = np.select(conditions_test, choices_test)
test_df.head()

test_ls = test_df.to_dict('records')

# Use to convert numeric value to Positive or Negative

In [ ]:
train_df_negative = train_df.loc[train_df['polarity of the tweet']==0]
train_df_positive = train_df.loc[train_df['polarity of the tweet']==4]

In [ ]:
# Pull evenly the amount of training set that will be used and combine them
frames = [train_df_negative.iloc[0:12500,:],train_df_positive.iloc[0:12500,:]]
result = pd.concat(frames)

In [ ]:
conditions = [
    (result['polarity of the tweet'] == 0),
    (result['polarity of the tweet'] == 2),
    (result['polarity of the tweet'] == 4)]
choices = ['negative', 'neutral', 'positive']
result['label'] = np.select(conditions, choices)
result.head()

# Convert dataframe to dictionary to run analysis

In [ ]:
train_ls = result.to_dict('records')

# Function to clean up the dataset

In [ ]:
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
import nltk
# nltk.download('punkt')

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]
    
tweetProcessor = PreProcessTweets()

In [ ]:
# Process Training Set
preprocessedTrainingSet = tweetProcessor.processTweets(train_ls)

In [ ]:
# Process Test Set
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

# Function to process NLTK

In [ ]:
import nltk 

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [ ]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

In [ ]:
word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)

# Execute only if model has to be retrained

In [ ]:
from nltk.classify import SklearnClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC

# SKLearnClassifier = SklearnClassifier(BernoulliNB()).train(trainingFeatures)
SKLearnClassifier = SklearnClassifier(SVC(), sparse=False).train(trainingFeatures)

# Compare the test set with the model

In [ ]:
SKLearnResultLabels = [SKLearnClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# Save the model to pickle for later use

In [ ]:
# Save Pickle
import pickle
save_classifier = open("sklearn_svc_train_25k.pickle","wb")
pickle.dump(SKLearnClassifier, save_classifier)
save_classifier.close()

# Use to load pickle

In [ ]:
# Load Pickle
import pickle
classifier_f = open("sklearn_svc_train_25k.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

# Compare the test set with the loaded pickle

In [ ]:
SKResultLabelsPickle = [classifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# Export results

In [ ]:
# had to add space before tweet text to eliminate csv issue with tweets that begins with @
# list comprehension
test_list_text = [' '+test['text'] for test in testDataSet]
# convert twitter time output to real time output
test_list_time = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(test['time'],'%a %b %d %H:%M:%S +0000 %Y')) 
for test in testDataSet]
test_list_search_term = [test['search_term'] for test in testDataSet]

In [ ]:
output_list = pd.DataFrame(
    {'Tweet': test_list_text,
     'Time': test_list_time,
     'Result': SKResultLabelsPickle,
     'Search_Term': test_list_search_term
    })

In [ ]:
output_list['Score']=np.where((output_list.Result=='negative'), -1, 1)
output_list.head()

In [ ]:
output_list

# Output to CSV file

In [ ]:
output_list.to_csv('./outputSKLearn_test.csv',header=True,
                              index=False,encoding="utf-8")